<a href="https://colab.research.google.com/github/aritanaoya/football_analysis/blob/main/work/J1Predict_by_%E5%BE%97%E5%A4%B1%E7%82%B9%E3%83%91%E3%82%BF%E3%83%BC%E3%83%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret==2.3.1

In [ ]:
from pycaret.utils import enable_colab

enable_colab()

Colab mode enabled.


In [ ]:
import numpy as np
import pandas as pd
import time

csv = '/content/scrapeData20211102.csv'
data = pd.read_csv(csv)
data.shape

(656, 24)

# データの加工

In [ ]:
dataList = ['ＰＫ','セットプレー直接','セットプレーから','クロスから','スルーパスから','ショートパスから','ロングパスから','ドリブルから','こぼれ球から','その他',
            'PK_A','セットプレー直接_A','セットプレーから_A','クロスから_A','スルーパスから_A','ショートパスから_A','ロングパスから_A','ドリブルから_A','こぼれ球から_A','その他_A']

dataList_all= [
               'Unnamed: 0','試合日','チーム名','スコア'] + dataList
for item in dataList:   
    data[item] = data[item].str.replace('(', '')
    data[item] = data[item].str.replace(')', '')
    data[item] = data[item].str.replace('%', '').astype(np.float64) / 100
data
df= pd.concat([data.iloc[::2].reset_index(drop=True), pd.DataFrame(
    data.iloc[1::2].values, columns=dataList_all)], 1)

In [ ]:
df

,Unnamed: 0,試合日,チーム名,スコア,ＰＫ,セットプレー直接,セットプレーから,クロスから,スルーパスから,ショートパスから,ロングパスから,ドリブルから,こぼれ球から,その他,PK_A,セットプレー直接_A,セットプレーから_A,クロスから_A,スルーパスから_A,ショートパスから_A,ロングパスから_A,ドリブルから_A,こぼれ球から_A,その他_A,Unnamed: 0,試合日,チーム名,スコア,ＰＫ,セットプレー直接,セットプレーから,クロスから,スルーパスから,ショートパスから,ロングパスから,ドリブルから,こぼれ球から,その他,PK_A,セットプレー直接_A,セットプレーから_A,クロスから_A,スルーパスから_A,ショートパスから_A,ロングパスから_A,ドリブルから_A,こぼれ球から_A,その他_A
0,home,2021-8-9-15:00,札幌,2,0.098,0.0,0.268,0.049,0.024,0.220,0.049,0.122,0.098,0.073,0.089,0.00,0.244,0.267,0.044,0.200,0.00,0.044,0.089,0.022,away,2021-8-9-15:00,浦和,1,0.159,0.023,0.136,0.182,0.114,0.205,0.068,0.045,0.045,0.023,0.029,0.059,0.147,0.235,0.029,0.176,0.088,0.029,0.118,0.088
1,home,2021-8-14-14:00,札幌,3,0.098,0.0,0.268,0.049,0.024,0.220,0.049,0.122,0.098,0.073,0.089,0.00,0.244,0.267,0.044,0.200,0.00,0.044,0.089,0.022,away,2021-8-14-14:00,FC東京,2,0.047,0.047,0.116,0.209,0.047,0.279,0.023,0.07,0.14,0.023,0,0,0.262,0.19,0.024,0.238,0.024,0.024,0.143,0.095
2,home,2021-8-28-14:00,札幌,0,0.098,0.0,0.268,0.049,0.024,0.220,0.049,0.122,0.098,0.073,0.089,0.00,0.244,0.267,0.044,0.200,0.00,0.044,0.089,0.022,away,2021-8-28-14:00,川崎Ｆ,2,0.043,0,0.1,0.314,0.086,0.257,0.014,0.043,0.043,0.1,0,0.095,0.095,0.333,0.048,0.095,0.048,0.095,0.19,0
3,home,2021-2-27-14:00,札幌,5,0.098,0.0,0.268,0.049,0.024,0.220,0.049,0.122,0.098,0.073,0.089,0.00,0.244,0.267,0.044,0.200,0.00,0.044,0.089,0.022,away,2021-2-27-14:00,横浜FC,1,0,0.097,0.323,0.161,0.097,0.161,0.065,0,0.097,0,0.085,0,0.254,0.197,0.099,0.113,0.014,0.056,0.155,0.028
4,home,2021-5-22-14:00,札幌,2,0.098,0.0,0.268,0.049,0.024,0.220,0.049,0.122,0.098,0.073,0.089,0.00,0.244,0.267,0.044,0.200,0.00,0.044,0.089,0.022,away,2021-5-22-14:00,清水,0,0,0,0.355,0.226,0,0.097,0.032,0.032,0.129,0.129,0.021,0.064,0.319,0.255,0.043,0.191,0,0.021,0.043,0.043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,home,2021-5-2-14:00,大分,1,0.083,0.0,0.292,0.167,0.000,0.125,0.000,0.083,0.250,0.000,0.041,0.02,0.122,0.286,0.061,0.265,0.02,0.041,0.082,0.061,away,2021-5-2-14:00,清水,0,0,0,0.355,0.226,0,0.097,0.032,0.032,0.129,0.129,0.021,0.064,0.319,0.255,0.043,0.191,0,0.021,0.043,0.043
324,home,2021-4-11-15:00,大分,0,0.083,0.0,0.292,0.167,0.000,0.125,0.000,0.083,0.250,0.000,0.041,0.02,0.122,0.286,0.061,0.265,0.02,0.041,0.082,0.061,away,2021-4-11-15:00,名古屋,3,0.054,0.027,0.216,0.243,0.027,0.108,0,0.108,0.162,0.054,0.077,0,0.269,0.154,0.077,0.269,0.077,0.077,0,0
325,home,2021-3-21-14:00,大分,1,0.083,0.0,0.292,0.167,0.000,0.125,0.000,0.083,0.250,0.000,0.041,0.02,0.122,0.286,0.061,0.265,0.02,0.041,0.082,0.061,away,2021-3-21-14:00,広島,3,0.105,0.053,0.263,0.105,0.105,0.132,0.053,0.105,0.079,0,0.091,0.03,0.182,0.182,0.061,0.242,0.061,0,0.152,0
326,home,2021-2-27-14:00,大分,1,0.083,0.0,0.292,0.167,0.000,0.125,0.000,0.083,0.250,0.000,0.041,0.02,0.122,0.286,0.061,0.265,0.02,0.041,0.082,0.061,away,2021-2-27-14:00,徳島,1,0.069,0,0.207,0.172,0.138,0.138,0.034,0,0.172,0.069,0.082,0.02,0.245,0.061,0.102,0.245,0.041,0.02,0.122,0.061


In [ ]:
filename =  'trainData' + '.csv'
df.to_csv(filename, encoding = 'utf-8-sig') 
data = pd.read_csv('/content/trainData.csv')
data.dtypes

Unnamed: 0        int64
Unnamed: 0.1     object
試合日              object
チーム名             object
スコア               int64
ＰＫ              float64
セットプレー直接        float64
セットプレーから        float64
クロスから           float64
スルーパスから         float64
ショートパスから        float64
ロングパスから         float64
ドリブルから          float64
こぼれ球から          float64
その他             float64
PK_A            float64
セットプレー直接_A      float64
セットプレーから_A      float64
クロスから_A         float64
スルーパスから_A       float64
ショートパスから_A      float64
ロングパスから_A       float64
ドリブルから_A        float64
こぼれ球から_A        float64
その他_A           float64
Unnamed: 0.2     object
試合日.1            object
チーム名.1           object
スコア.1             int64
ＰＫ.1            float64
セットプレー直接.1      float64
セットプレーから.1      float64
クロスから.1         float64
スルーパスから.1       float64
ショートパスから.1      float64
ロングパスから.1       float64
ドリブルから.1        float64
こぼれ球から.1        float64
その他.1           float64
PK_A.1          float64
セットプレー直接_A.1    float64
セットプレーから_A.1    

# SetUP

In [ ]:
from pycaret.regression import setup
data = data.drop('チーム名', axis=1)
def x(a,b):
    return a - b
data['スコア']  = data.apply(lambda f: x(f['スコア'],f['スコア.1']), axis=1)
data = data.drop('スコア.1', axis=1)
data = data.drop('Unnamed: 0',axis = 1)
data = data.drop('Unnamed: 0.1',axis = 1)
data = data.drop('Unnamed: 0.2',axis = 1)
setup = setup(data, target='スコア',
        session_id = 123)

,Description,Value
0,session_id,123
1,Target,スコア
2,Original Data,"(328, 44)"
3,Missing Values,False
4,Numeric Features,40
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(229, 86)"


In [ ]:
setup

(0       1
 1       1
 2      -2
 3       4
 4       2
        ..
 323     1
 324    -3
 325    -2
 326     0
 327     0
 Name: スコア, Length: 328, dtype: object,
 False,
 False,
 {'USI',
  'X',
  'X_test',
  'X_train',
  '_all_metrics',
  '_all_models',
  '_all_models_internal',
  '_available_plots',
  '_gpu_n_jobs_param',
  '_internal_pipeline',
  '_ml_usecase',
  'create_model_container',
  'data_before_preprocess',
  'display_container',
  'exp_name_log',
  'experiment__',
  'fix_imbalance_method_param',
  'fix_imbalance_param',
  'fold_generator',
  'fold_groups_param',
  'fold_groups_param_full',
  'fold_param',
  'fold_shuffle_param',
  'gpu_param',
  'html_param',
  'imputation_classifier',
  'imputation_regressor',
  'iterative_imputation_iters_param',
  'log_plots_param',
  'logging_param',
  'master_model_container',
  'n_jobs_param',
  'prep_pipe',
  'pycaret_globals',
  'seed',
  'stratify_param',
  'target_param',
  'transform_target_method_param',
  'transform_target_param

In [ ]:
from pycaret.regression import compare_models

best_model = compare_models()  # 全てのモデルを訓練し、評価する

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,1.315500e+00,2.818100e+00,1.672800e+00,5.270000e-02,0.0,8.922000e-01,0.573
ada,AdaBoost Regressor,1.351200e+00,2.896000e+00,1.691200e+00,2.430000e-02,0.0,9.248000e-01,0.120
lightgbm,Light Gradient Boosting Machine,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.016
omp,Orthogonal Matching Pursuit,1.391800e+00,3.112600e+00,1.755900e+00,-4.790000e-02,0.0,9.689000e-01,0.016
lasso,Lasso Regression,1.387500e+00,3.177300e+00,1.775400e+00,-5.580000e-02,0.0,9.882000e-01,0.016
en,Elastic Net,1.387500e+00,3.177300e+00,1.775400e+00,-5.580000e-02,0.0,9.882000e-01,0.018
llar,Lasso Least Angle Regression,1.387500e+00,3.177300e+00,1.775400e+00,-5.580000e-02,0.0,9.882000e-01,0.017
br,Bayesian Ridge,1.381000e+00,3.178000e+00,1.775900e+00,-5.870000e-02,0.0,9.762000e-01,0.030
et,Extra Trees Regressor,1.407300e+00,3.173500e+00,1.772700e+00,-7.100000e-02,0.0,9.748000e-01,0.514
gbr,Gradient Boosting Regressor,1.412000e+00,3.218800e+00,1.786200e+00,-7.830000e-02,0.0,9.761000e-01,0.125


In [ ]:
print(best_model)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=123, verbose=0, warm_start=False)


# Create Model

In [ ]:
from pycaret.regression import create_model

model = create_model("rf")  # Bayesian Ridgeのモデルを作成

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1.1378,2.8721,1.6947,-0.1493,-0.0,0.6240
1,1.4630,3.1382,1.7715,0.2983,-0.0,0.8264
2,1.1774,2.3418,1.5303,0.0105,-0.0,0.8796
3,1.3539,2.9941,1.7303,-0.0545,-0.0,0.9622
4,1.1922,2.3204,1.5233,0.2515,-0.0,0.8949
5,1.2396,2.9144,1.7072,0.0205,-0.0,0.8682
6,1.1765,1.9160,1.3842,0.2569,-0.0,0.9309
7,1.4461,2.9903,1.7292,0.0259,-0.0,1.0034
8,1.4309,3.1711,1.7808,-0.0781,-0.0,1.0072
9,1.5377,3.5230,1.8770,-0.0552,-0.0,0.9252


# Tune Model

In [ ]:
from pycaret.regression import tune_model

tuned_model = tune_model(model)  # ハイパーパラメータの調整

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1.1840,2.7442,1.6566,-0.0981,-0.0,0.7252
1,1.5062,3.4144,1.8478,0.2366,-0.0,0.8726
2,1.0905,2.2160,1.4886,0.0637,-0.0,0.8221
3,1.2205,2.6608,1.6312,0.0629,-0.0,0.8281
4,1.1729,2.4078,1.5517,0.2234,-0.0,0.8837
5,1.2828,2.9299,1.7117,0.0153,-0.0,0.9073
6,1.1424,1.9840,1.4085,0.2305,-0.0,0.9486
7,1.3883,2.7416,1.6558,0.1070,-0.0,0.9433
8,1.4580,3.3061,1.8183,-0.1240,-0.0,1.0650
9,1.4119,3.0298,1.7406,0.0926,-0.0,0.8741


In [ ]:
print(tuned_model)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=6, max_features='log2', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0002,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=290, n_jobs=-1, oob_score=False,
                      random_state=123, verbose=0, warm_start=False)


# evalation

In [ ]:
from pycaret.regression import evaluate_model

evaluate_model(tuned_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

# deploy

In [ ]:
from pycaret.regression import finalize_model

final_model = finalize_model(tuned_model)
print(final_model)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=6, max_features='log2', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0002,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=290, n_jobs=-1, oob_score=False,
                      random_state=123, verbose=0, warm_start=False)


#前回
	Model	MAE	MSE	RMSE	R2	RMSLE	MAPE
0	Bayesian Ridge	0.8964	1.2683	1.1262	0.1359	0.5328	0.3625

#今回
問題点＝＞列ごとの予測になっていて相性的なところが見れていない（チーム毎にどこが相手でも同じスコアになってしまう）


Model	MAE	MSE	RMSE	R2	RMSLE	MAPE
0	Bayesian Ridge	0.8013	1.1055	1.0514	0.1474	0.4946	0.3233

In [ ]:
from pycaret.regression import predict_model

predict_model(final_model);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,0.8592,1.2463,1.1164,0.4947,0.4875,0.6458


In [ ]:
from pycaret.regression import save_model

save_model(final_model, "save_model")

Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='スコア',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy='me...
                  RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                        criterion='mse', max_depth=6,
                                        max_features='log2', max_leaf_nodes=None,
                                        max_samples=None,
                                

In [ ]:
from pycaret.regression import load_model

loaded_model = load_model("save_model")

Transformation Pipeline and Model Successfully Loaded


# get Pred Data

In [ ]:
#試合日は以下を例に入力
#2021-8-9-15:00
def scrapePredData(homeTeamID,awayTeamID,year,matchDate):
    data = pd.DataFrame()
    url = 'https://www.football-lab.jp/comparison/team/' + year + '/' + homeTeamID + '/' + year + '/' + awayTeamID
    #得点パターン、失点パターンを取得
    df1 = pd.read_html(url)[18]
    df1 = df1.drop([0,2,4,6,8,10,12,14,16,18]).drop(df1.columns[[0,2,4,6]], axis=1)


    df2 = pd.read_html(url)[19]
    df2 = df2.drop([0,2,4,6,8,10,12,14,16,18]).drop(df2.columns[[0,2,4,6]], axis=1)
    df2 = df2.replace({'ＰＫ':'PK_A','セットプレー直接':'セットプレー直接_A','セットプレーから':'セットプレーから_A','クロスから':'クロスから_A',
                       'スルーパスから':'スルーパスから_A','ショートパスから':'ショートパスから_A','ロングパスから':'ロングパスから_A','ドリブルから':'ドリブルから_A',
                       'こぼれ球から':'こぼれ球から_A','その他':'その他_A'})

    df = pd.concat([df1,df2])
    df = df.reset_index(drop=True)
    df = df.rename(columns={'割合': 'home','Unnamed: 3': 'Data','割合.1': 'away'})
    df = df[['Data', 'home', 'away']]
    df = df.set_index('Data')
    df = df.T

    time.sleep(1)
    #チーム名と試合日
    homeTeam = get_keys_from_value(TeamDic,homeTeamID)
    awayTeam = get_keys_from_value(TeamDic,awayTeamID)
    df.insert(0, 'チーム名',[homeTeam,awayTeam])
    df.insert(0, '試合日', matchDate)
    data = pd.concat([data, df]) 
    dataList = ['ＰＫ','セットプレー直接','セットプレーから','クロスから','スルーパスから','ショートパスから','ロングパスから','ドリブルから','こぼれ球から','その他',
            'PK_A','セットプレー直接_A','セットプレーから_A','クロスから_A','スルーパスから_A','ショートパスから_A','ロングパスから_A','ドリブルから_A','こぼれ球から_A','その他_A']
    for item in dataList:   
        data[item] = data[item].str.replace('(', '')
        data[item] = data[item].str.replace(')', '')
        data[item] = data[item].str.replace('%', '').astype(np.float64) / 100

    return data


### Team Dic

In [ ]:
TeamDic = {'札幌':'276', '仙台':'238', '鹿島':'120', '浦和':'122', '柏':'132', 'FC東京':'270', '川崎Ｆ':'86', '横浜FM':'124', '横浜FC':'296' , '湘南':'130', '清水':'126', '名古屋':'127', 'Ｇ大阪':'128','Ｃ大阪':'133', '神戸':'136','広島':'129', '徳島':'30116', '福岡':'135', '鳥栖':'269', '大分':'207'}

def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]

### ここのTeamDicと日付を変えれば予想できる

In [ ]:
pred_data = scrapePredData(TeamDic['札幌'],TeamDic['仙台'],'2021','2021-11-03-13:00')
dataList_pred = [ '試合日','チーム名'] + dataList
pred_df= pd.concat([pred_data.iloc[::2].reset_index(drop=True), pd.DataFrame(
    pred_data.iloc[1::2].values, columns=dataList_pred)], 1)
pred_df

,試合日,チーム名,ＰＫ,セットプレー直接,セットプレーから,クロスから,スルーパスから,ショートパスから,ロングパスから,ドリブルから,こぼれ球から,その他,PK_A,セットプレー直接_A,セットプレーから_A,クロスから_A,スルーパスから_A,ショートパスから_A,ロングパスから_A,ドリブルから_A,こぼれ球から_A,その他_A,試合日,チーム名,ＰＫ,セットプレー直接,セットプレーから,クロスから,スルーパスから,ショートパスから,ロングパスから,ドリブルから,こぼれ球から,その他,PK_A,セットプレー直接_A,セットプレーから_A,クロスから_A,スルーパスから_A,ショートパスから_A,ロングパスから_A,ドリブルから_A,こぼれ球から_A,その他_A
0,2021-11-03-13:00,[札幌],0.091,0.0,0.273,0.068,0.023,0.227,0.045,0.114,0.091,0.068,0.083,0.0,0.25,0.292,0.042,0.188,0.0,0.042,0.083,0.021,2021-11-03-13:00,[仙台],0,0,0.448,0.207,0.034,0.103,0,0.069,0.103,0.034,0.053,0.035,0.263,0.14,0.07,0.193,0.018,0.07,0.105,0.053


In [ ]:
filename =  'predData' + '.csv'
pred_df.to_csv(filename, encoding = 'utf-8-sig') 
pred_data = pd.read_csv('/content/predData.csv')
pred_data.dtypes

Unnamed: 0        int64
試合日              object
チーム名             object
ＰＫ              float64
セットプレー直接        float64
セットプレーから        float64
クロスから           float64
スルーパスから         float64
ショートパスから        float64
ロングパスから         float64
ドリブルから          float64
こぼれ球から          float64
その他             float64
PK_A            float64
セットプレー直接_A      float64
セットプレーから_A      float64
クロスから_A         float64
スルーパスから_A       float64
ショートパスから_A      float64
ロングパスから_A       float64
ドリブルから_A        float64
こぼれ球から_A        float64
その他_A           float64
試合日.1            object
チーム名.1           object
ＰＫ.1            float64
セットプレー直接.1      float64
セットプレーから.1      float64
クロスから.1         float64
スルーパスから.1       float64
ショートパスから.1      float64
ロングパスから.1       float64
ドリブルから.1        float64
こぼれ球から.1        float64
その他.1           float64
PK_A.1          float64
セットプレー直接_A.1    float64
セットプレーから_A.1    float64
クロスから_A.1       float64
スルーパスから_A.1     float64
ショートパスから_A.1    float64
ロングパスから_A.1     

In [ ]:
pred_data['チーム名'].astype(str)
pred_data['チーム名'] = pred_data['チーム名'].str.replace('[', '')
pred_data['チーム名'] = pred_data['チーム名'].str.replace('\'', '')
pred_data['チーム名'] = pred_data['チーム名'].str.replace(']', '')
pred_data['チーム名.1'] = pred_data['チーム名.1'].str.replace('[', '')
pred_data['チーム名.1'] = pred_data['チーム名.1'].str.replace('\'', '')
pred_data['チーム名.1'] = pred_data['チーム名.1'].str.replace(']', '')

pred_data

,Unnamed: 0,試合日,チーム名,ＰＫ,セットプレー直接,セットプレーから,クロスから,スルーパスから,ショートパスから,ロングパスから,ドリブルから,こぼれ球から,その他,PK_A,セットプレー直接_A,セットプレーから_A,クロスから_A,スルーパスから_A,ショートパスから_A,ロングパスから_A,ドリブルから_A,こぼれ球から_A,その他_A,試合日.1,チーム名.1,ＰＫ.1,セットプレー直接.1,セットプレーから.1,クロスから.1,スルーパスから.1,ショートパスから.1,ロングパスから.1,ドリブルから.1,こぼれ球から.1,その他.1,PK_A.1,セットプレー直接_A.1,セットプレーから_A.1,クロスから_A.1,スルーパスから_A.1,ショートパスから_A.1,ロングパスから_A.1,ドリブルから_A.1,こぼれ球から_A.1,その他_A.1
0,0,2021-11-03-13:00,札幌,0.091,0.0,0.273,0.068,0.023,0.227,0.045,0.114,0.091,0.068,0.083,0.0,0.25,0.292,0.042,0.188,0.0,0.042,0.083,0.021,2021-11-03-13:00,仙台,0.0,0.0,0.448,0.207,0.034,0.103,0.0,0.069,0.103,0.034,0.053,0.035,0.263,0.14,0.07,0.193,0.018,0.07,0.105,0.053


# prediction

In [ ]:

# pred_data = pred_data.drop('チーム名', axis=1)
pred_data = pred_data.drop('Unnamed: 0',axis = 1)
prediction = predict_model(loaded_model, data=pred_data)


In [ ]:
print('スコア',prediction['Label'][0])

スコア 0.3799959831381432
